Create a Model that Predicts Overall Value

In [19]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [20]:

data = pd.read_csv('fifa21_train.csv')


# Rename Columns

In [21]:
data.columns = data.columns.str.lower().str.replace(' ','_')
data.columns = data.columns.str.lower().str.replace('/','_')

In [22]:
data = data.rename(columns={'bp':'best_position',
                            'team_&_contract':'team_contract', 
                            'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })

In [23]:
data.to_csv('fifa_cleaned3.csv')

# Checking

In [24]:
# data.head()
# data.columns.tolist()
# data1.dtypes.tolist()
# data.isna().count().tolist()
# data['height']

# New DF

#### Clean Wage

In [25]:
icon = '€'

data['wage'] = data['wage'].str.replace(icon, '')
data['wage'] = data['wage'].str.replace('K', '000')

data['wage'].astype('int')

0         4000
1        23000
2        49000
3         4000
4         2000
         ...  
11696     3000
11697     4000
11698     2000
11699        0
11700     2000
Name: wage, Length: 11701, dtype: int32

#### Divide Columns, Choose Right

In [26]:
data_pos = data.loc[:, 'ls':'gk']

for col in data_pos.columns:
    data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)

# Convertir las columnas resultantes en numéricas
data_pos = data_pos.astype(int)

data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)

#### Drop Columns

In [27]:
data_att = data.loc[:, 'crossing':'gk_reflexes']
data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)

#### Clean Star

In [28]:
icon = '★'

data['international_reputation'] = data['international_reputation'].str.replace(icon, '')
data['international_reputation'] = data['international_reputation'].str.replace(' ', '')

data['international_reputation'].astype('int')

0        1
1        2
2        2
3        1
4        1
        ..
11696    1
11697    1
11698    1
11699    3
11700    1
Name: international_reputation, Length: 11701, dtype: int32

#### Categorical to Numerical

In [29]:
data_cat = pd.DataFrame(data['best_position'])

encoder = OneHotEncoder(drop='first').fit(data_cat)
encoded = encoder.transform(data_cat).toarray()

data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

#### Clean NaN

In [30]:
# data_att.describe()

In [31]:
for col in data_att.columns:
    data_att[col] = data_att[col].fillna(data_att[col].mean())

# data_att.describe()

#### Concatenate Numerical

In [32]:
data_num = pd.concat([data_att, data_pos, data['international_reputation'], data['wage']], axis=1)
data_num.columns.tolist()
# data_nor.isna().sum().tolist()

['crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'fk_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'composure',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'ls_right',
 'st_right',
 'rs_right',
 'lw_right',
 'lf_right',
 'cf_right',
 'rf_right',
 'rw_right',
 'lam_right',
 'cam_right',
 'ram_right',
 'lm_right',
 'lcm_right',
 'cm_right',
 'rcm_right',
 'rm_right',
 'lwb_right',
 'ldm_right',
 'cdm_right',
 'rdm_right',
 'rwb_right',
 'lb_right',
 'lcb_right',
 'cb_right',
 'rcb_right',
 'rb_right',
 'gk_right',
 'international_reputation',
 'wage']

# Check Correlation

In [33]:
data1_target = pd.concat([data['ova'], data_num], axis=1)
data1_target
correlation_matrix = data1_target.corr()
correlation_matrix

C:\Users\PC\AppData\Local\Temp\ipykernel_17056\3092851269.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data1_target.corr()


,ova,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,...,ldm_right,cdm_right,rdm_right,rwb_right,lb_right,lcb_right,cb_right,rcb_right,rb_right,gk_right
ova,1.000000,0.390354,0.306632,0.304702,0.493152,0.360994,0.350262,0.399692,0.372304,0.479454,...,-0.008443,-0.008443,-0.008443,0.025408,0.063187,-0.046561,-0.046561,-0.046561,0.063187,0.040295
crossing,0.390354,1.000000,0.645621,0.435570,0.800162,0.674937,0.854544,0.831791,0.751382,0.740585,...,-0.085794,-0.085794,-0.085794,-0.105374,-0.047227,-0.001007,-0.001007,-0.001007,-0.047227,0.143883
finishing,0.306632,0.645621,1.000000,0.455388,0.650934,0.888847,0.820629,0.760857,0.695429,0.485792,...,-0.037664,-0.037664,-0.037664,-0.019330,-0.005118,0.088715,0.088715,0.088715,-0.005118,0.128136
heading_accuracy,0.304702,0.435570,0.455388,1.000000,0.630159,0.489435,0.531864,0.413944,0.366814,0.478891,...,-0.072665,-0.072665,-0.072665,-0.026832,-0.025207,-0.217723,-0.217723,-0.217723,-0.025207,0.157763
short_passing,0.493152,0.800162,0.650934,0.630159,1.000000,0.683227,0.839028,0.764904,0.719229,0.886005,...,-0.106755,-0.106755,-0.106755,-0.049393,-0.008647,-0.064731,-0.064731,-0.064731,-0.008647,0.180633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lcb_right,-0.046561,-0.001007,0.088715,-0.217723,-0.064731,0.045441,0.055077,0.033770,-0.010705,-0.105413,...,0.647764,0.647764,0.647764,0.509886,0.572440,1.000000,1.000000,1.000000,0.572440,0.451816
cb_right,-0.046561,-0.001007,0.088715,-0.217723,-0.064731,0.045441,0.055077,0.033770,-0.010705,-0.105413,...,0.647764,0.647764,0.647764,0.509886,0.572440,1.000000,1.000000,1.000000,0.572440,0.451816
rcb_right,-0.046561,-0.001007,0.088715,-0.217723,-0.064731,0.045441,0.055077,0.033770,-0.010705,-0.105413,...,0.647764,0.647764,0.647764,0.509886,0.572440,1.000000,1.000000,1.000000,0.572440,0.451816
rb_right,0.063187,-0.047227,-0.005118,-0.025207,-0.008647,-0.025835,-0.005715,-0.028331,-0.054342,-0.024910,...,0.736960,0.736960,0.736960,0.921632,1.000000,0.572440,0.572440,0.572440,1.000000,0.762873


# Normalize

In [34]:
tra = MinMaxScaler().fit(data_num)
nor = tra.transform(data_num)

data_nor = pd.DataFrame(nor, columns = data_num.columns)

data_nor

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,...,rdm_right,rwb_right,lb_right,lcb_right,cb_right,rcb_right,rb_right,gk_right,international_reputation,wage
0,0.545455,0.478261,0.431818,0.720930,0.465116,0.615385,0.444444,0.561798,0.642857,0.637363,...,0.750,0.666667,0.666667,0.750,0.750,0.750,0.666667,0.333333,0.00,0.007143
1,0.681818,0.826087,0.806818,0.697674,0.837209,0.857143,0.822222,0.752809,0.642857,0.813187,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.25,0.041071
2,0.761364,0.793478,0.329545,0.813953,0.825581,0.879121,0.944444,0.966292,0.773810,0.879121,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.25,0.087500
3,0.431818,0.423913,0.602273,0.627907,0.372093,0.538462,0.411111,0.460674,0.571429,0.615385,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.00,0.007143
4,0.488636,0.369565,0.636364,0.697674,0.348837,0.648352,0.444444,0.449438,0.619048,0.670330,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.00,0.003571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.636364,0.228261,0.522727,0.593023,0.279070,0.571429,0.500000,0.269663,0.535714,0.593407,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.00,0.005357
11697,0.068182,0.119565,0.090909,0.244186,0.139535,0.131868,0.122222,0.157303,0.214286,0.142857,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.00,0.007143
11698,0.659091,0.684783,0.522727,0.651163,0.430233,0.725275,0.588889,0.370787,0.583333,0.681319,...,0.875,0.833333,0.833333,0.875,0.875,0.875,0.833333,0.666667,0.00,0.003571
11699,0.659091,0.684783,0.522727,0.755814,0.477527,0.758242,0.506865,0.651685,0.750000,0.769231,...,0.625,0.500000,0.500000,0.625,0.625,0.625,0.500000,0.000000,0.50,0.000000


# Concatenate

In [35]:
data1 = pd.concat([data_cat, data_nor], axis=1)
# data1.describe().T
# data1.dtypes

# Train-Test Split

In [36]:
X = pd.concat([data1], axis=1)
y = data['ova']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

# Linear Regression

In [37]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [38]:
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.899349984203019

# R2

In [39]:
predictions_test = lm.predict(X_test) # The R2 test should be similar to the train test. That means is a strong model.
r2_score(y_test, predictions_test)

0.8931438085577318

# MAE

In [40]:
mae = mean_absolute_error(y_test, predictions_test) # It is under 2, so it´s good. We can also compare this with the range of the target. It should be similar to the RMSE. But the RMSE is going to be bigger because of the squaring.
mae

1.6947886746685157

# MSE

In [41]:
mse=mean_squared_error(y_test,predictions_test)
mse

4.842112109585015

# RMSE

In [42]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test)) # usually bigger than MAE. It is affected by outliers.
rmse

2.200479972548038

# Create a DEF Function

In [43]:
def clean (df):

    # Rename columns
    df.columns = df.columns.str.lower().str.replace(' ','_')
    df.columns = df.columns.str.lower().str.replace('/','_')
    df = df.rename(columns={'bp':'best_position',
                                'team_&_contract':'team_contract', 
                                'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })
    # Edit wage   
    icon = '€'

    df['wage'] = df['wage'].str.replace(icon, '')
    df['wage'] = df['wage'].str.replace('K', '000')

    df['wage'].astype('int')

    # Edit positions
    data_pos = df.loc[:, 'ls':'gk']
    for col in data_pos.columns:
        data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)
    data_pos = data_pos.astype(int)
    data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)
    
    # Edit attributes
    data_att = df.loc[:, 'crossing':'gk_reflexes']
    data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)
    
    # Edit international reputation
    icon = '★'
    df['international_reputation'] = df['international_reputation'].str.replace(icon, '')
    df['international_reputation'] = df['international_reputation'].str.replace(' ', '')
    df['international_reputation'].astype('int')
    
    # Edit best position
    data_cat = pd.DataFrame(df['best_position'])
    # don´t use this, it´s already trained : encoder = OneHotEncoder(drop='first').fit(data_cat)
    encoded = encoder.transform(data_cat).toarray()
    data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())
    
    # Fill na
    for col in data_att.columns:
        data_att[col] = data_att[col].fillna(data_att[col].mean())
    
    # Concat
    data_num = pd.concat([data_att, data_pos, df['international_reputation'], df['wage']], axis=1)
    
    
    # Normalise
    # don´t use this, it´s already trained : tra = MinMaxScaler().fit(data_num)
    nor = tra.transform(data_num)
    data_nor = pd.DataFrame(nor, columns = data_num.columns)
    
    # concatenate final
    df = pd.concat([data_cat, data_nor], axis=1)
    
    return df

In [44]:
data_validate = pd.read_csv('fifa21_validate.csv')

In [45]:
data_validate_clean = clean(data_validate)

In [46]:
X_val = data_validate_clean
y_val = data_validate['ova']

predictions_test_val = lm.predict(X_val)
r2_score(y_val, predictions_test_val)

0.8975628473307108

In [47]:
mae = mean_absolute_error(y_val, predictions_test_val)
mae

1.6766146111773996

In [48]:
mse=mean_squared_error(y_val,predictions_test_val)
mse

4.687675819735937

In [49]:
rmse = np.sqrt(mse)
rmse

2.1651041129091086